<a href="https://colab.research.google.com/github/cxz260/XzUtil/blob/main/ImageClip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

# Load the pre-trained ResNet50 model with ImageNet weights
model = ResNet50(weights='imagenet')


102967424/102967424 [==============================] - 1s 0us/step


In [4]:
image_path = "/content/Mytree.jpg"

# Load and preprocess your image
img = image.load_img(image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Make a prediction
predictions = model.predict(img_array)

# Decode the prediction
decoded_predictions = decode_predictions(predictions, top=5)[0]

print("Predicted classifications:")
for i, (imagenet_id, label, prob) in enumerate(decoded_predictions):
    print(f"{i+1}. {label}: {prob * 100:.2f}%")


35363/35363 [==============================] - 0s 0us/step
Predicted classifications:
1. pole: 39.38%
2. walking_stick: 24.82%
3. cabbage_butterfly: 13.59%
4. picket_fence: 2.78%
5. greenhouse: 2.10%


2. Using CLIP model


In [ ]:
pip install torch torchvision ftfy

In [ ]:
!rm -rf CLIP
!git clone https://github.com/openai/CLIP.git
!pip install -e ./CLIP

In [21]:
import sys
sys.path.append('/content/CLIP')

In [22]:
import torch
import torchvision.transforms as T
from PIL import Image
from ftfy import fix_text
from torch.nn.functional import softmax
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:07<00:00, 44.6MiB/s]


In [24]:
image_path = "/content/Mytree.jpg"
image = Image.open(image_path).convert("RGB")
image_input = preprocess(image).unsqueeze(0).to(device)
tree_species = [
    "acer palmatum", "silver birch", "scots pine", "norway spruce", "english oak",
    "weeping willow", "american elm", "sugar maple", "colorado blue spruce",
    "black walnut", "white oak", "douglas fir", "giant sequoia", "quaking aspen"
]

text_input = clip.tokenize([fix_text(species) for species in tree_species]).to(device)
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_input)
    logits = image_features @ text_features.T
    probs = softmax(logits, dim=-1)

probs = probs.cpu().numpy().flatten()
predicted_species = tree_species[probs.argmax()]
print(f"Predicted tree species: {predicted_species} (probabilities: {probs})")


Predicted tree species: white oak (probabilities: [1.04222854e-03 3.00771004e-04 1.07295895e-02 8.89795925e-03
 1.60680208e-02 1.09218262e-01 2.42957354e-01 1.95050508e-01
 4.96348366e-04 6.09508455e-02 3.38462621e-01 1.17816767e-02
 2.16734243e-05 4.02205391e-03])
